In [1]:
import math
import re
import sys
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve,roc_auc_score

%config Warning.simplerfilter = True

# device =torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:0')
device = torch.device('cpu')

In [2]:
import random
seed_value = 42

torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## 初始数据集的特征

In [3]:
ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始GRU所有特征-1.csv')
ori_fea = ori_fea.drop(ori_fea.columns[0], axis=1)

label = [1] * 8000 + [0] * 8000

oy = pd.DataFrame(label, columns=['label'])
ori_fea

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.380952,315,0.346032,0.155556,0.225397,0.273016,0.169329,0.015974,0.089457,0.070288,...,0.151716,0.151716,0.151716,0.151716,0.151716,0.151716,0.151716,0.151716,0.151716,0.151716
1,0.409396,596,0.261745,0.177852,0.231544,0.328859,0.104377,0.042088,0.055556,0.060606,...,0.214509,0.214509,0.214509,0.214509,0.214509,0.214509,0.214509,0.214509,0.214509,0.214509
2,0.343101,819,0.312576,0.191697,0.151404,0.344322,0.105263,0.059976,0.048960,0.097919,...,0.232799,0.232799,0.232799,0.232799,0.232799,0.232799,0.232799,0.232799,0.232799,0.232799
3,0.461774,327,0.244648,0.299694,0.162080,0.293578,0.080000,0.058462,0.036923,0.067692,...,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533
4,0.565217,23,0.130435,0.347826,0.217391,0.304348,0.047619,0.000000,0.000000,0.095238,...,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.450256,975,0.274872,0.217436,0.232821,0.274872,0.069887,0.045221,0.073998,0.085303,...,0.317155,0.311166,0.424684,0.298339,0.333873,0.345730,0.345730,0.315714,0.266020,0.354227
15996,0.430957,1557,0.289017,0.185613,0.245344,0.280026,0.093248,0.041801,0.079100,0.074598,...,0.319268,0.396904,0.461847,0.398506,0.344183,0.266691,0.237441,0.244302,0.313226,0.261287
15997,0.455010,2934,0.311179,0.190184,0.264826,0.233810,0.106753,0.042633,0.103342,0.058322,...,0.325447,0.304716,0.375515,0.316105,0.421821,0.336021,0.237929,0.337070,0.255288,0.306693
15998,0.448404,2694,0.293615,0.213066,0.235338,0.257981,0.089525,0.056092,0.080609,0.067608,...,0.324376,0.307927,0.404629,0.281485,0.320470,0.265394,0.358318,0.253283,0.285617,0.451122


In [22]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'seqname': headers, 'seq': sequences}
    df = pd.DataFrame(data)
    return df
file_path = r'../Data/TestSets/cRNA/Cicer arietinum.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'../Data/TestSets/ncRNA/Cicer arietinum.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'
datah['seqname'] = [re.search(pattern, identifier).group(1) for identifier in datah['seqname']]

data = datah
datah

2099
2099


,seqname,seq
0,Ca_05183,ATGAGAGAAATCCTCCACATTCAGGGCGGGCAGTGCGGTAACCAGA...
1,Ca_24212,ATGGCCGAAGCTATCATTGGTTTCACCGTTGTCAAGTCACATATTC...
2,Ca_07518,ATGGGTGGCAATGGTAAGCATAGGTGGAAAATTTCCTTCCATCGTT...
3,Ca_07783,ATGAGGCCTGACTTGAGTTTGCAAGACATAAGAAACAGGATATTTC...
4,Ca_21854,ATGAGACTCCAAATTCAATCGCTTGTTCTTCTCTTACTTGTTCTCT...
...,...,...
4193,URS0000DB64E7_3827,TCAGCAGTGATGAGCCTCTAACATATAAGTAGAATCCCTATGTGGG...
4194,URS0000DB654B_3827,AAAAAAGCAGTTGTCACCTTTGTGAACAGTCCTCTTGTAACCTTCA...
4195,URS0000DB68DD_3827,GTCTCCGATGATGAACAATAAAATATAACTTTCGTTCTTCTGAGTT...
4196,URS0000DB692E_3827,TTGCACGCAGTATCGTAGGTGTCTCTTATAATATAGAGCGATCTTG...


In [23]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'../Features/TestSets_cds_features/c_cds_test/Cicer_arietinum',names=names,sep='\t')
print(rna_cds.shape[0])
rna_cds_nc = pd.read_csv(r'../Features/TestSets_cds_features/nc_cds_test/Cicer_arietinum',names=names,sep='\t')
print(rna_cds_nc.shape[0])

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)

datah_seqname = datah['seqname']
rna_cds_seqname = rna_cds['seqname']

# # extra_rows_index = datah_seqname.difference(rna_cds_seqname)
indexes_to_drop = datah_seqname[~datah_seqname.isin(rna_cds_seqname)].index

datah.drop(indexes_to_drop, inplace=True)

label_data = [0] * 2099 + [1] * 1957

y = pd.DataFrame(label_data, columns=['label'])
datah.reset_index(drop=True, inplace=True)

datah.insert(0,'label',y)
print(datah.index)
print(y.index)

data3 = datah.copy()
datah

2099
1957
RangeIndex(start=0, stop=4056, step=1)
RangeIndex(start=0, stop=4056, step=1)


,label,seqname,seq
0,0,Ca_05183,ATGAGAGAAATCCTCCACATTCAGGGCGGGCAGTGCGGTAACCAGA...
1,0,Ca_24212,ATGGCCGAAGCTATCATTGGTTTCACCGTTGTCAAGTCACATATTC...
2,0,Ca_07518,ATGGGTGGCAATGGTAAGCATAGGTGGAAAATTTCCTTCCATCGTT...
3,0,Ca_07783,ATGAGGCCTGACTTGAGTTTGCAAGACATAAGAAACAGGATATTTC...
4,0,Ca_21854,ATGAGACTCCAAATTCAATCGCTTGTTCTTCTCTTACTTGTTCTCT...
...,...,...,...
4051,1,URS0000DB64E7_3827,TCAGCAGTGATGAGCCTCTAACATATAAGTAGAATCCCTATGTGGG...
4052,1,URS0000DB654B_3827,AAAAAAGCAGTTGTCACCTTTGTGAACAGTCCTCTTGTAACCTTCA...
4053,1,URS0000DB68DD_3827,GTCTCCGATGATGAACAATAAAATATAACTTTCGTTCTTCTGAGTT...
4054,1,URS0000DB692E_3827,TTGCACGCAGTATCGTAGGTGTCTCTTATAATATAGAGCGATCTTG...


## 手动特征

In [6]:
from itertools import product
# compute k-mers features (k=1~4)
def Kmers_funct(seq, x):
    X = [None] * len(seq)
    for i in range(len(seq)):
        a = seq[i]
        t = 0
        l = []
        for index in range(len(a) - x + 1):
            t = a[index:index + x]
            if (len(t)) == x:
                l.append(t)
        X[i] = l
    return X


def nucleotide_type(k):
    z = []
    for i in product('ACGT', repeat=k):
        z.append(''.join(i))
    return z


def Kmers_frequency(seq, x):
    X = []
    char = nucleotide_type(x)
    for i in range(len(seq)):
        s = seq[i]
        frequence = []
        for a in char:
            number = s.count(a)
            char_frequence = number / (len(s) - x + 1)
            frequence.append(char_frequence)
        X.append(frequence)
    return X


feature_1mer = Kmers_funct(data.seq, 1)  # 1-mer
feature_2mer = Kmers_funct(data.seq, 2)  # 2-mer
feature_3mer = Kmers_funct(data.seq, 3)  # 3-mer

feature_1mer_frequency = Kmers_frequency(feature_1mer, 1)  # 1-mer
feature_2mer_frequency = Kmers_frequency(feature_2mer, 2)  # 2-mer
feature_3mer_frequency = Kmers_frequency(feature_3mer, 3)  # 3-mer

feature = pd.concat([pd.DataFrame(feature_1mer_frequency),
                     pd.DataFrame(feature_2mer_frequency),
                     pd.DataFrame(feature_3mer_frequency)], axis=1)
feature.columns = ['A','C','G','T','AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT','AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
feature

,A,C,G,T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.254682,0.208989,0.259925,0.276404,0.060015,0.050263,0.068267,0.075769,0.068267,0.042011,...,0.010518,0.022539,0.028550,0.018032,0.024793,0.020285,0.012772,0.027799,0.022539,0.016529
1,0.335116,0.156863,0.253119,0.254902,0.110912,0.048301,0.093023,0.082290,0.075134,0.026834,...,0.000000,0.017953,0.025135,0.001795,0.032316,0.016158,0.014363,0.016158,0.028725,0.023339
2,0.254603,0.219048,0.267937,0.258413,0.078195,0.054037,0.069930,0.052765,0.060394,0.040687,...,0.019096,0.019733,0.021006,0.010821,0.019096,0.015913,0.014640,0.024188,0.022915,0.024825
3,0.364055,0.168971,0.204301,0.262673,0.174114,0.040062,0.066256,0.083205,0.060092,0.040062,...,0.003091,0.018547,0.026275,0.015456,0.023184,0.017002,0.010819,0.020093,0.023184,0.020093
4,0.280112,0.235294,0.184874,0.299720,0.090141,0.067606,0.036620,0.084507,0.059155,0.047887,...,0.022663,0.028329,0.022663,0.005666,0.008499,0.019830,0.019830,0.036827,0.011331,0.022663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,0.255556,0.200000,0.222222,0.322222,0.056818,0.022727,0.079545,0.102273,0.056818,0.045455,...,0.000000,0.034884,0.046512,0.000000,0.023256,0.023256,0.011628,0.034884,0.011628,0.000000
4052,0.264000,0.200000,0.208000,0.328000,0.130081,0.056911,0.048780,0.032520,0.056911,0.065041,...,0.000000,0.016529,0.033058,0.008264,0.016529,0.049587,0.008264,0.024793,0.057851,0.041322
4053,0.307692,0.153846,0.219780,0.318681,0.123596,0.056180,0.056180,0.078652,0.011236,0.022472,...,0.011494,0.045977,0.057471,0.011494,0.011494,0.011494,0.011494,0.045977,0.022989,0.045977
4054,0.274390,0.207317,0.182927,0.335366,0.030864,0.049383,0.080247,0.117284,0.092593,0.018519,...,0.006250,0.037500,0.012500,0.025000,0.006250,0.018750,0.031250,0.031250,0.012500,0.018750


In [7]:
def gc_content(dna_sequence):
    gc_percent = []
    for i in dna_sequence:
        gc_count = i.count('G') + i.count('C')
        total_count = len(i)
        gc_percent_eve = (gc_count / total_count) * 100
        gc_percent.append(gc_percent_eve)
    return gc_percent

In [8]:
# gc_precent = gc_content(data.iloc[:,0])
gc_precent = gc_content(data.seq)
gc_precent_pd = pd.DataFrame(gc_precent)
gc_precent_pd.rename(columns={0: 'GC content'}, inplace=True)
print(gc_precent_pd.shape)
gc_precent_pd

feature.insert(0,'GC content',gc_precent_pd)

feature = pd.DataFrame(feature)
feature.iloc[:,0] = feature.iloc[:,0]/100
feature

(4056, 1)


,GC content,A,C,G,T,AA,AC,AG,AT,CA,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.468914,0.254682,0.208989,0.259925,0.276404,0.060015,0.050263,0.068267,0.075769,0.068267,...,0.010518,0.022539,0.028550,0.018032,0.024793,0.020285,0.012772,0.027799,0.022539,0.016529
1,0.409982,0.335116,0.156863,0.253119,0.254902,0.110912,0.048301,0.093023,0.082290,0.075134,...,0.000000,0.017953,0.025135,0.001795,0.032316,0.016158,0.014363,0.016158,0.028725,0.023339
2,0.486984,0.254603,0.219048,0.267937,0.258413,0.078195,0.054037,0.069930,0.052765,0.060394,...,0.019096,0.019733,0.021006,0.010821,0.019096,0.015913,0.014640,0.024188,0.022915,0.024825
3,0.373272,0.364055,0.168971,0.204301,0.262673,0.174114,0.040062,0.066256,0.083205,0.060092,...,0.003091,0.018547,0.026275,0.015456,0.023184,0.017002,0.010819,0.020093,0.023184,0.020093
4,0.420168,0.280112,0.235294,0.184874,0.299720,0.090141,0.067606,0.036620,0.084507,0.059155,...,0.022663,0.028329,0.022663,0.005666,0.008499,0.019830,0.019830,0.036827,0.011331,0.022663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,0.422222,0.255556,0.200000,0.222222,0.322222,0.056818,0.022727,0.079545,0.102273,0.056818,...,0.000000,0.034884,0.046512,0.000000,0.023256,0.023256,0.011628,0.034884,0.011628,0.000000
4052,0.408000,0.264000,0.200000,0.208000,0.328000,0.130081,0.056911,0.048780,0.032520,0.056911,...,0.000000,0.016529,0.033058,0.008264,0.016529,0.049587,0.008264,0.024793,0.057851,0.041322
4053,0.373626,0.307692,0.153846,0.219780,0.318681,0.123596,0.056180,0.056180,0.078652,0.011236,...,0.011494,0.045977,0.057471,0.011494,0.011494,0.011494,0.011494,0.045977,0.022989,0.045977
4054,0.390244,0.274390,0.207317,0.182927,0.335366,0.030864,0.049383,0.080247,0.117284,0.092593,...,0.006250,0.037500,0.012500,0.025000,0.006250,0.018750,0.031250,0.031250,0.012500,0.018750


In [9]:
sequence_len = []
for i in range(data.shape[0]):
    sequence_len.append(len(data.seq[i]))
sequence_len = pd.DataFrame(sequence_len)
sequence_len.rename(columns={0:'seq_len'},inplace=True)
sequence_len

feature.insert(1,'seq_len',sequence_len)
feature.rename(columns={'GC content':'GC_con'},inplace=True)
print(feature.shape)
feature

(4056, 86)


,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.468914,1335,0.254682,0.208989,0.259925,0.276404,0.060015,0.050263,0.068267,0.075769,...,0.010518,0.022539,0.028550,0.018032,0.024793,0.020285,0.012772,0.027799,0.022539,0.016529
1,0.409982,561,0.335116,0.156863,0.253119,0.254902,0.110912,0.048301,0.093023,0.082290,...,0.000000,0.017953,0.025135,0.001795,0.032316,0.016158,0.014363,0.016158,0.028725,0.023339
2,0.486984,1575,0.254603,0.219048,0.267937,0.258413,0.078195,0.054037,0.069930,0.052765,...,0.019096,0.019733,0.021006,0.010821,0.019096,0.015913,0.014640,0.024188,0.022915,0.024825
3,0.373272,651,0.364055,0.168971,0.204301,0.262673,0.174114,0.040062,0.066256,0.083205,...,0.003091,0.018547,0.026275,0.015456,0.023184,0.017002,0.010819,0.020093,0.023184,0.020093
4,0.420168,357,0.280112,0.235294,0.184874,0.299720,0.090141,0.067606,0.036620,0.084507,...,0.022663,0.028329,0.022663,0.005666,0.008499,0.019830,0.019830,0.036827,0.011331,0.022663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,0.422222,90,0.255556,0.200000,0.222222,0.322222,0.056818,0.022727,0.079545,0.102273,...,0.000000,0.034884,0.046512,0.000000,0.023256,0.023256,0.011628,0.034884,0.011628,0.000000
4052,0.408000,125,0.264000,0.200000,0.208000,0.328000,0.130081,0.056911,0.048780,0.032520,...,0.000000,0.016529,0.033058,0.008264,0.016529,0.049587,0.008264,0.024793,0.057851,0.041322
4053,0.373626,91,0.307692,0.153846,0.219780,0.318681,0.123596,0.056180,0.056180,0.078652,...,0.011494,0.045977,0.057471,0.011494,0.011494,0.011494,0.011494,0.045977,0.022989,0.045977
4054,0.390244,164,0.274390,0.207317,0.182927,0.335366,0.030864,0.049383,0.080247,0.117284,...,0.006250,0.037500,0.012500,0.025000,0.006250,0.018750,0.031250,0.031250,0.012500,0.018750


## cds特征

In [10]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'../Features/TestSets_cds_features/c_cds_test/Cicer_arietinum',names=names,sep='\t', index_col=index_col)
rna_cds = rna_cds
rna_cds_nc = pd.read_csv(r'../Features/TestSets_cds_features/nc_cds_test/Cicer_arietinum',names=names,sep='\t', index_col=index_col)
rna_cds_nc = rna_cds_nc

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)
rna_cds

data1 = data
data2 = data
for i, l in data2.iterrows():
    data1.iloc[i,2] = len(l['seq'])
    
mylist = []
mylist = pd.DataFrame(mylist,columns=['score','cdsStarts','cdsStop','cdsSizes','cdsPercent'])

for index, row in rna_cds.iterrows():
    tem_pd_score = pd.DataFrame(row)
    mylist.loc[index, 'cdsStarts'] = row["start"]
    mylist.loc[index, 'cdsStop'] = row["end"]
    mylist.loc[index, 'cdsSizes'] = row["end"] - row["start"]
    mylist.loc[index, 'score'] = tem_pd_score.iloc[4, 0]
    mylist.loc[index, 'cdsPercent'] = (row["start"] + row["end"]) / data1.loc[index, 'seq']
    
feature = feature.join(mylist)
feature

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TGT,TTA,TTC,TTG,TTT,score,cdsStarts,cdsStop,cdsSizes,cdsPercent
0,0.468914,1335,0.254682,0.208989,0.259925,0.276404,0.060015,0.050263,0.068267,0.075769,...,0.020285,0.012772,0.027799,0.022539,0.016529,1435.0,0,1335,1335,1.0
1,0.409982,561,0.335116,0.156863,0.253119,0.254902,0.110912,0.048301,0.093023,0.082290,...,0.016158,0.014363,0.016158,0.028725,0.023339,761.0,0,561,561,1.0
2,0.486984,1575,0.254603,0.219048,0.267937,0.258413,0.078195,0.054037,0.069930,0.052765,...,0.015913,0.014640,0.024188,0.022915,0.024825,1775.0,0,1575,1575,1.0
3,0.373272,651,0.364055,0.168971,0.204301,0.262673,0.174114,0.040062,0.066256,0.083205,...,0.017002,0.010819,0.020093,0.023184,0.020093,751.0,0,651,651,1.0
4,0.420168,357,0.280112,0.235294,0.184874,0.299720,0.090141,0.067606,0.036620,0.084507,...,0.019830,0.019830,0.036827,0.011331,0.022663,457.0,0,357,357,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,0.422222,90,0.255556,0.200000,0.222222,0.322222,0.056818,0.022727,0.079545,0.102273,...,0.023256,0.011628,0.034884,0.011628,0.000000,212.0,9,21,12,0.333333
4052,0.408000,125,0.264000,0.200000,0.208000,0.328000,0.130081,0.056911,0.048780,0.032520,...,0.049587,0.008264,0.024793,0.057851,0.041322,157.5,114,123,9,1.896
4053,0.373626,91,0.307692,0.153846,0.219780,0.318681,0.123596,0.056180,0.056180,0.078652,...,0.011494,0.011494,0.045977,0.022989,0.045977,216.5,10,28,18,0.417582
4054,0.390244,164,0.274390,0.207317,0.182927,0.335366,0.030864,0.049383,0.080247,0.117284,...,0.018750,0.031250,0.031250,0.012500,0.018750,245.0,104,149,45,1.542683


## 深度特征

In [11]:
sequences = data3['seq']
labels = data3['label'].values
print(labels)

pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)

word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])

text = x.apply(lambda x: [word_index.get(word, 0) for word in x])
print(text)

text_len = 1200

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]
pad_text = np.array(pad_text)
pad_text

pad_text,labels=torch.LongTensor(pad_text),torch.LongTensor(labels)
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)
print(x_train.shape)
print(y_train.shape)
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list

    def __getitem__(self, index):
        text = torch.LongTensor(self.text_list[index])
        label = self.label_list[index]
        return text, label

    def __len__(self):
        return len(self.text_list)

data_ds = Mydataset(pad_text,labels)

batch_size = 16

data_dl =torch.utils.data.DataLoader(data_ds,batch_size=batch_size,shuffle=False)

# embed_dim = 2 ** (int(np.log2(len(word_list) ** 0.25)) + 2) 

embed_dim = 50
hidden_size = 20

class Net(nn.Module):
    def __init__(self, word_list, embed_dim, hidden_size, num_layers=2):
        super().__init__()

        self.em = nn.Embedding(len(word_list) + 1, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_size, num_layers)
        self.pool1=nn.MaxPool1d(5)
        self.pool2 = nn.MaxPool1d(3)
        self.linear1 = nn.Linear(hidden_size, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, inputs):
        x = self.em(inputs)  # inputs is input, size (seq_len, batch, input_size)               #[16,1200,50]
        x = x.float()
        x, _ = self.gru(x)  # x is outuput, size (seq_len, batch, hidden_size)                  #[16,1200,20]
        x = self.pool1(x)                                                                       #[16,1200,4]
        x =self.pool2(x)                                                                        #[16,1200,1]
        x =x.view(-1,1200)
        return x

model = Net(word_list, embed_dim, hidden_size)                  #实例化
model = model.to(device)

loss = nn.CrossEntropyLoss()
loss = loss.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

with torch.no_grad():
    outputs=[]
    for x,y in data_dl:
        out = model(x)
        outputs.append(out)

outputs1 = torch.concat(outputs,axis =0)

deep_fea=pd.DataFrame(outputs1)
deep_fea                                  

[0 0 0 ... 1 1 1]
0       [3, 2, 1, 3, 1, 3, 1, 3, 3, 3, 2, 4, 4, 2, 4, ...
1       [3, 2, 1, 1, 4, 4, 1, 3, 3, 1, 4, 2, 3, 2, 4, ...
2       [3, 2, 1, 1, 1, 2, 1, 1, 4, 3, 3, 2, 1, 1, 2, ...
3       [3, 2, 1, 3, 1, 1, 4, 4, 2, 1, 3, 4, 2, 2, 1, ...
4       [3, 2, 1, 3, 1, 3, 4, 2, 4, 4, 3, 3, 3, 2, 2, ...
                              ...                        
4051    [2, 4, 3, 1, 4, 3, 1, 2, 1, 3, 2, 1, 3, 1, 4, ...
4052    [3, 3, 3, 3, 3, 3, 1, 4, 3, 1, 2, 2, 1, 2, 4, ...
4053    [1, 2, 4, 2, 4, 4, 1, 3, 2, 1, 3, 2, 1, 3, 3, ...
4054    [2, 2, 1, 4, 3, 4, 1, 4, 3, 1, 2, 3, 2, 4, 1, ...
4055    [4, 1, 2, 4, 2, 1, 2, 1, 3, 2, 1, 3, 2, 3, 2, ...
Name: seq, Length: 4056, dtype: object
torch.Size([2839, 1200])
torch.Size([2839])


,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.127456,0.196661,0.176346,0.127456,0.176346,0.127456,0.176346,0.127456,0.127456,0.127456,...,0.176346,0.196661,0.127456,0.176350,0.127456,0.176350,0.196661,0.176346,0.176346,0.196661
1,0.221215,0.317528,0.286162,0.190498,0.148730,0.278371,0.286162,0.221215,0.221215,0.190498,...,0.240297,0.206610,0.179649,0.182971,0.179649,0.182971,0.206610,0.240297,0.240297,0.206610
2,0.280880,0.378655,0.340816,0.306597,0.272369,0.212499,0.340816,0.213890,0.339966,0.222035,...,0.172013,0.192267,0.218907,0.289426,0.272254,0.289426,0.323979,0.332191,0.172013,0.192267
3,0.331839,0.406452,0.367519,0.249964,0.354654,0.330891,0.244576,0.279128,0.248793,0.292206,...,0.242821,0.265283,0.244658,0.321644,0.301791,0.321644,0.306274,0.299064,0.242821,0.265283
4,0.366547,0.418220,0.377760,0.276029,0.390193,0.284691,0.274357,0.252974,0.298825,0.237398,...,0.265598,0.281151,0.261817,0.308957,0.287541,0.308957,0.273329,0.258181,0.265598,0.281151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,0.284221,0.223055,0.213113,0.377166,0.279128,0.276824,0.334919,0.260441,0.371616,0.255120,...,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533
4052,0.224715,0.291561,0.292853,0.287257,0.365642,0.328072,0.390740,0.220591,0.294706,0.298504,...,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506
4053,0.319571,0.250135,0.413654,0.298679,0.455120,0.405629,0.423512,0.291884,0.312054,0.351997,...,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691
4054,0.348060,0.276646,0.353734,0.236877,0.496105,0.520416,0.427250,0.390013,0.222532,0.370895,...,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562


In [12]:
fea_all = feature.join(deep_fea)

y = pd.DataFrame(label_data, columns=['label'])

fea_all.insert(0,'label',y)
fea_all

,label,GC_con,seq_len,A,C,G,T,AA,AC,AG,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0,0.468914,1335,0.254682,0.208989,0.259925,0.276404,0.060015,0.050263,0.068267,...,0.176346,0.196661,0.127456,0.176350,0.127456,0.176350,0.196661,0.176346,0.176346,0.196661
1,0,0.409982,561,0.335116,0.156863,0.253119,0.254902,0.110912,0.048301,0.093023,...,0.240297,0.206610,0.179649,0.182971,0.179649,0.182971,0.206610,0.240297,0.240297,0.206610
2,0,0.486984,1575,0.254603,0.219048,0.267937,0.258413,0.078195,0.054037,0.069930,...,0.172013,0.192267,0.218907,0.289426,0.272254,0.289426,0.323979,0.332191,0.172013,0.192267
3,0,0.373272,651,0.364055,0.168971,0.204301,0.262673,0.174114,0.040062,0.066256,...,0.242821,0.265283,0.244658,0.321644,0.301791,0.321644,0.306274,0.299064,0.242821,0.265283
4,0,0.420168,357,0.280112,0.235294,0.184874,0.299720,0.090141,0.067606,0.036620,...,0.265598,0.281151,0.261817,0.308957,0.287541,0.308957,0.273329,0.258181,0.265598,0.281151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4051,1,0.422222,90,0.255556,0.200000,0.222222,0.322222,0.056818,0.022727,0.079545,...,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533,0.237533
4052,1,0.408000,125,0.264000,0.200000,0.208000,0.328000,0.130081,0.056911,0.048780,...,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506,0.275506
4053,1,0.373626,91,0.307692,0.153846,0.219780,0.318681,0.123596,0.056180,0.056180,...,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691,0.303691
4054,1,0.390244,164,0.274390,0.207317,0.182927,0.335366,0.030864,0.049383,0.080247,...,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562,0.324562


In [ ]:
### RFC trainning model

def model_RFC(X, y, X_test, y_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict

    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)

    rf_clf = RandomForestClassifier()

    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    y_pred = cross_val_predict(rf_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    rf_clf.fit(X, y)
    y_pred_test = rf_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")

model_RFC(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

In [ ]:
# Naive Bayes trainning model
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

def model_gnb(X, y, X_test, y_test):
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    gnb = GaussianNB()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行五倍交叉验证
    cv_results = cross_validate(gnb, X, y, scoring=scoring, cv=5, return_train_score=False)

    # 计算并输出评估指标
    # print("Accuracy: %f (+/- %f)" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std() * 2))
    # print("F1 score: %f (+/- %f)" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std() * 2))
    # print("ROC AUC score: %f (+/- %f)" % (cv_results['test_roc_auc'].mean(), cv_results['test_roc_auc'].std() * 2))
    # print("MCC: %f (+/- %f)" % (cv_results['test_mcc'].mean(), cv_results['test_mcc'].std() * 2))

    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(gnb, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    gnb.fit(X, y)
    y_pred_test = gnb.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_gnb(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

In [ ]:
# SVM trainning model

def model_SVM(X, y, X_test, y_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建SVM分类器
    svm_clf = SVC()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(svm_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(svm_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    svm_clf.fit(X, y)
    y_pred_test = svm_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_SVM(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

In [15]:
### RFC训练模型

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
from sklearn.model_selection import cross_val_predict

X, y = ori_fea,oy

X.columns = X.columns.astype(str)

    
# 创建随机森林分类器
rf_clf = RandomForestClassifier()

# 定义评估指标
scoring = {'accuracy': 'accuracy',
        'f1': 'f1',
        'roc_auc': 'roc_auc',
        'mcc': make_scorer(matthews_corrcoef)}

# 进行5倍交叉验证并计算评估指标
cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

# 输出评估指标的平均值和标准差
print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
print("F1 score: %f" % (cv_results['test_f1'].mean()))
print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
print("MCC: %f" % (cv_results['test_mcc'].mean()))

# 计算混淆矩阵
y_pred = cross_val_predict(rf_clf, X, y, cv=5)
rf_clf.fit(X, y)
tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
npv = tn / (tn + fn)
ppv = tp / (tp + fp)
se = tp / (tp + fn)
spc = tn / (tn + fp)

# 输出混淆矩阵及相关指标
print(f"NPV: {npv:f}")
print(f"PPV: {ppv:f}")
print(f"Sensitivity (Recall): {se:f}")
print(f"Specificity: {spc:f}")

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

Accuracy: 0.982187
F1 score: 0.982396
ROC AUC score: 0.996161
MCC: 0.964646


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionW

NPV: 0.994241
PPV: 0.971781
Sensitivity (Recall): 0.994375
Specificity: 0.971125


In [17]:
import joblib

# 保存模型为.pkl文件
joblib.dump(rf_clf, 'model.pkl')

['model.pkl']

In [13]:
import joblib
# 读取模型的方法
rf_clf = joblib.load('model.pkl')

In [14]:
X, y, X_test, y_test = ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0]

# 在独立测试集上评估模型

X_test.columns = X_test.columns.astype(str)
y_pred_test = rf_clf.predict(X_test)
auc_score_test = roc_auc_score(y_test, y_pred_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1_score_test = f1_score(y_test, y_pred_test)
mcc_test = matthews_corrcoef(y_test, y_pred_test)

# 输出独立测试集上的评估指标
print("Test Set - Accuracy: %f" % (accuracy_test))
print("Test Set - F1 score: %f" % (f1_score_test))
print("Test Set - ROC AUC score: %f" % (auc_score_test))
print("Test Set - MCC: %f" % (mcc_test))
    
# 计算混淆矩阵和评估指标（独立测试集）
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
npv_test = tn_test / (tn_test + fn_test)
ppv_test = tp_test / (tp_test + fp_test)
se_test = tp_test / (tp_test + fn_test)
spc_test = tn_test / (tn_test + fp_test)

# 输出混淆矩阵及相关指标（独立测试集）
print(f"Test Set - NPV: {npv_test:f}")
print(f"Test Set - PPV: {ppv_test:f}")
print(f"Test Set - Sensitivity (Recall): {se_test:f}")
print(f"Test Set - Specificity: {spc_test:f}")

Test Set - Accuracy: 0.941815
Test Set - F1 score: 0.943133
Test Set - ROC AUC score: 0.943783
Test Set - MCC: 0.889972
Test Set - NPV: 1.000000
Test Set - PPV: 0.892385
Test Set - Sensitivity (Recall): 1.000000
Test Set - Specificity: 0.887566
